<a href="https://colab.research.google.com/github/Uniholder/Specialization-Yandex-MIPT/blob/main/6_Final_projects/2_churn_prediction/week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir data
!wget -O data/orange_small_churn_data.train https://stepik.org/media/attachments/lesson/90608/_847b6cd041836e8fa914243e83d898fb_orange_small_churn_data.train
!wget -O data/orange_small_churn_labels.train https://stepik.org/media/attachments/lesson/90608/_ce5e9e4280a8856218f1886f007de2a4_orange_small_churn_labels.train

--2022-08-12 17:26:49--  https://stepik.org/media/attachments/lesson/90608/_847b6cd041836e8fa914243e83d898fb_orange_small_churn_data.train
Resolving stepik.org (stepik.org)... 13.69.75.239
Connecting to stepik.org (stepik.org)|13.69.75.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23722578 (23M) [application/octet-stream]
Saving to: ‘data/orange_small_churn_data.train’

data/orange_small_c 100%[===================>]  22.62M  14.9MB/s    in 1.5s    

2022-08-12 17:26:51 (14.9 MB/s) - ‘data/orange_small_churn_data.train’ saved [23722578/23722578]

--2022-08-12 17:26:51--  https://stepik.org/media/attachments/lesson/90608/_ce5e9e4280a8856218f1886f007de2a4_orange_small_churn_labels.train
Resolving stepik.org (stepik.org)... 13.69.75.239
Connecting to stepik.org (stepik.org)|13.69.75.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117024 (114K) [application/octet-stream]
Saving to: ‘data/orange_small_churn_labels.train’

data/ora

# Входные данные

In [2]:
import pandas as pd
import os
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

PATH_TO_DATA = 'data'

In [3]:
df_raw = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_data.train'))
labels = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_labels.train'), header=None, squeeze=True)
df_raw['target'] = labels
df_raw.shape

(40000, 231)

In [4]:
num_vars = df_raw.columns[:190]
cat_vars = df_raw.columns[190:]

In [5]:
df = df_raw.dropna(axis=1, how='all')
df.shape

(40000, 213)

Отделим hold-out dataset

In [6]:
df_train, df_ho = train_test_split(df, test_size=5000)
df_train.shape, df_ho.shape

((35000, 213), (5000, 213))

In [7]:
# df_ho.to_csv(os.path.join(PATH_TO_DATA, 'hold_out_dataset.csv'))

Preprocessing

In [8]:
cols = df_train.drop('target', axis=1).columns

In [10]:
%%time
si = SimpleImputer(strategy='most_frequent')
df_train_imp = pd.DataFrame(si.fit_transform(df_train[cols]), columns=cols)

CPU times: user 2.05 s, sys: 559 ms, total: 2.61 s
Wall time: 2.64 s


In [11]:
oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df_train_enc = pd.DataFrame(oe.fit_transform(df_train_imp), columns=cols)

In [12]:
df_train_enc['Var229'].value_counts()

0.0    28072
1.0     6874
3.0       28
2.0       26
Name: Var229, dtype: int64

# Модель

In [13]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.1 MB/s 


In [35]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, cross_validate
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report

from catboost import CatBoostClassifier, cv, Pool

In [15]:
X, y = df_train_enc, df_train['target']
cv = StratifiedShuffleSplit(random_state=0, )

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

## LogisticRegression

In [20]:
cv_results = cross_validate(estimator=LogisticRegression(random_state=0), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv)

In [21]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.57955681, 0.56908052, 0.57392911, 0.53710245, 0.54516159,
        0.52890749, 0.56647276, 0.54868784, 0.53080881, 0.54142568]),
 0.5521133069420635)

In [33]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
print(classification_report(y_val, lr.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      6495
           1       0.00      0.00      0.00       505

    accuracy                           0.93      7000
   macro avg       0.46      0.50      0.48      7000
weighted avg       0.86      0.93      0.89      7000



## RidgeClassifier

In [22]:
cv_results = cross_validate(estimator=RidgeClassifier(random_state=0), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv)

In [23]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.67753053, 0.68228501, 0.69095053, 0.663883  , 0.66253206,
        0.65549624, 0.63695723, 0.64087184, 0.66371145, 0.63083156]),
 0.6605049444913684)

In [38]:
rc = RidgeClassifier(random_state=0)
rc.fit(X_train, y_train)
print(classification_report(y_val, rc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      6495
           1       0.00      0.00      0.00       505

    accuracy                           0.93      7000
   macro avg       0.46      0.50      0.48      7000
weighted avg       0.86      0.93      0.89      7000



## SGDClassifier

In [24]:
cv_results = cross_validate(estimator=SGDClassifier(random_state=0), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv)

In [25]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.48057883, 0.50599522, 0.55066421, 0.48439933, 0.56892922,
        0.56165634, 0.59028924, 0.50995033, 0.47393137, 0.49646601]),
 0.5222860097281572)

In [41]:
sgdc = SGDClassifier(random_state=0)
sgdc.fit(X_train, y_train)
print(classification_report(y_val, sgdc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      6495
           1       0.00      0.00      0.00       505

    accuracy                           0.93      7000
   macro avg       0.46      0.50      0.48      7000
weighted avg       0.86      0.93      0.89      7000



### SGDClassifier log loss

In [26]:
cv_results = cross_validate(estimator=SGDClassifier(random_state=0, loss='log'), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv)

In [27]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.47791389, 0.49289806, 0.50395452, 0.48042277, 0.47256495,
        0.49413582, 0.47998199, 0.49019143, 0.5302489 , 0.50054264]),
 0.4922854974690828)

In [61]:
sgdc = SGDClassifier(random_state=0, loss='log')
sgdc.fit(X_train, y_train)
print(classification_report(y_val, sgdc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.93      0.73      0.82      6495
           1       0.08      0.30      0.13       505

    accuracy                           0.70      7000
   macro avg       0.51      0.52      0.47      7000
weighted avg       0.87      0.70      0.77      7000



## RandomForestClassifier

In [28]:
rfc = RandomForestClassifier(oob_score=True)

In [29]:
rfc.fit(X, y)

RandomForestClassifier(oob_score=True)

In [30]:
rfc.oob_score_

0.9259714285714286

In [31]:
%%time
cv_results = cross_validate(estimator=RandomForestClassifier(random_state=0), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv)

CPU times: user 845 ms, sys: 180 ms, total: 1.03 s
Wall time: 1min 56s


In [32]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.66380913, 0.68038965, 0.66716562, 0.65747023, 0.68408685,
        0.65940252, 0.68314155, 0.62481431, 0.64669849, 0.65660892]),
 0.6623587266907229)

In [66]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)
print(classification_report(y_val, rfc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      6495
           1       0.00      0.00      0.00       505

    accuracy                           0.93      7000
   macro avg       0.46      0.50      0.48      7000
weighted avg       0.86      0.93      0.89      7000



## GradientBoostingClassifier

In [33]:
%%time
cv_results = cross_validate(estimator=GradientBoostingClassifier(random_state=0), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv)

CPU times: user 2.72 s, sys: 321 ms, total: 3.04 s
Wall time: 7min 17s


In [34]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.74443454, 0.73950077, 0.73361337, 0.71810383, 0.73966994,
        0.72455055, 0.72940689, 0.71341011, 0.73133084, 0.72711066]),
 0.7301131496904406)

In [18]:
gbc = GradientBoostingClassifier(random_state=0)
gbc.fit(X_train, y_train)
print(classification_report(y_val, gbc.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      6474
           1       0.20      0.00      0.01       526

    accuracy                           0.92      7000
   macro avg       0.56      0.50      0.48      7000
weighted avg       0.87      0.92      0.89      7000



## CatBoost

In [38]:
base_params = {
    'eval_metric': 'AUC:hints=skip_train~false', 
    'auto_class_weights': 'Balanced', 
    'verbose': False,
    'loss_function': 'Logloss'
}

In [41]:
cv_results = cv(
    params={**base_params},
    pool=Pool(X, y),
    fold_count=5,
    shuffle=True,
    plot=True,
    type='Classical'
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 0.7253457451
bestIteration = 482

Training on fold [1/5]

bestTest = 0.7235990012
bestIteration = 443

Training on fold [2/5]

bestTest = 0.7459748945
bestIteration = 392

Training on fold [3/5]

bestTest = 0.7117074429
bestIteration = 174

Training on fold [4/5]

bestTest = 0.720332754
bestIteration = 418



In [42]:
cv_results

,iterations,test-AUC-mean,test-AUC-std,train-AUC-mean,train-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.656049,0.010923,0.663241,0.007030,0.690138,0.000325,0.689971,0.000293
1,1,0.673163,0.007873,0.683537,0.008147,0.687167,0.000389,0.686745,0.000284
2,2,0.680584,0.011540,0.691241,0.007511,0.684291,0.000623,0.683701,0.000359
3,3,0.686526,0.010849,0.697513,0.006930,0.681539,0.000589,0.680765,0.000394
4,4,0.687200,0.011649,0.698605,0.005960,0.679300,0.000715,0.678323,0.000559
...,...,...,...,...,...,...,...,...,...
995,995,0.713295,0.013221,0.966828,0.000874,0.661487,0.019771,0.365987,0.002656
996,996,0.713334,0.013245,0.966915,0.000879,0.661542,0.019769,0.365789,0.002624
997,997,0.713343,0.013241,0.966997,0.000873,0.661621,0.019798,0.365593,0.002603
998,998,0.713315,0.013186,0.967095,0.000864,0.661726,0.019742,0.365399,0.002634


In [43]:
cv_results['test-AUC-mean'].argmax(), cv_results['test-AUC-mean'].max()

(442, 0.7237255172923177)

# Тест

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
df_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_test_data.csv'), index_col=0)
df_test

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
ID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,1225.0,7.0,NaN,NaN,NaN,...,zCkv,APgdzOv,jySVZNlOJy,NaN,ELof,xb3V,6fzt,Zy3gnGM,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,896.0,14.0,NaN,NaN,NaN,...,oslk,IIvC99a,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,791.0,7.0,NaN,NaN,NaN,...,oslk,6YSocsg,LM8l689qOp,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,mj86,NaN
3,NaN,NaN,NaN,NaN,NaN,2296.0,7.0,NaN,NaN,NaN,...,oslk,5nQ7A2G,jySVZNlOJy,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,am7c,NaN
4,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,...,oslk,MI8s5nE,LM8l689qOp,NaN,NaN,7P5s,RAYp,F2FyR07IdsN7I,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,714.0,7.0,NaN,NaN,NaN,...,zCkv,P6pu4Vl,LM8l689qOp,NaN,ELof,xb3V,ZI9m,R4y5gQQWY8OodqDV,am7c,NaN
9996,NaN,NaN,NaN,NaN,NaN,812.0,7.0,NaN,NaN,NaN,...,oslk,sXbT3Cb,LM8l689qOp,NaN,ELof,WqMG,RAYp,55YFVY9,mj86,NaN
9997,NaN,NaN,NaN,NaN,NaN,819.0,7.0,NaN,NaN,NaN,...,oslk,05jAV0N,M_8D,NaN,ELof,TNEC,RAYp,55YFVY9,am7c,NaN


In [ ]:
df_test.drop(cols_nan, axis=1, inplace=True)

In [ ]:
df_test = si.transform(df_test)
df_test = pd.DataFrame(df_test, columns=cols)

In [ ]:
df_test

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0.0,0.0,0.0,0.0,0.0,1225.0,7.0,0.0,0.0,8.0,...,UF16siJ,zCkv,APgdzOv,jySVZNlOJy,4n2X,ELof,xb3V,6fzt,Zy3gnGM,am7c
1,0.0,0.0,0.0,0.0,0.0,896.0,14.0,0.0,0.0,8.0,...,ot6oLzk,oslk,IIvC99a,LM8l689qOp,4n2X,ELof,xb3V,RAYp,F2FyR07IdsN7I,am7c
2,0.0,0.0,0.0,0.0,0.0,791.0,7.0,0.0,0.0,8.0,...,6VLNqhB,oslk,6YSocsg,LM8l689qOp,4n2X,kG3k,rgKb,RAYp,F2FyR07IdsN7I,mj86
3,0.0,0.0,0.0,0.0,0.0,2296.0,7.0,0.0,0.0,8.0,...,tzp8jNM,oslk,5nQ7A2G,jySVZNlOJy,4n2X,kG3k,rgKb,RAYp,F2FyR07IdsN7I,am7c
4,8.0,0.0,0.0,0.0,0.0,0.0,7.0,28.0,0.0,8.0,...,EPImS85,oslk,MI8s5nE,LM8l689qOp,4n2X,ELof,7P5s,RAYp,F2FyR07IdsN7I,am7c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,714.0,7.0,0.0,0.0,8.0,...,ch2oGfM,zCkv,P6pu4Vl,LM8l689qOp,4n2X,ELof,xb3V,ZI9m,R4y5gQQWY8OodqDV,am7c
9996,0.0,0.0,0.0,0.0,0.0,812.0,7.0,0.0,0.0,8.0,...,8d5YWu3,oslk,sXbT3Cb,LM8l689qOp,4n2X,ELof,WqMG,RAYp,55YFVY9,mj86
9997,0.0,0.0,0.0,0.0,0.0,819.0,7.0,0.0,0.0,8.0,...,WdPoLzk,oslk,05jAV0N,M_8D,4n2X,ELof,TNEC,RAYp,55YFVY9,am7c
9998,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,8.0,...,EMvv4zN,oslk,RDY7kpB,LM8l689qOp,4n2X,ELof,uWr3,RAYp,55YFVY9,am7c


In [ ]:
df_train_part

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0.0,0.0,0.0,0.0,171.0,0.0,1.0,0.0,134.0,0.0,...,2606.0,4.0,1781.0,0.0,0.0,0.0,19.0,2.0,7.0,0.0
1,0.0,0.0,0.0,0.0,0.0,73.0,0.0,0.0,0.0,0.0,...,3251.0,3.0,2892.0,0.0,0.0,0.0,13.0,4.0,7.0,0.0
2,0.0,0.0,0.0,0.0,0.0,122.0,1.0,0.0,0.0,0.0,...,3342.0,4.0,1023.0,0.0,0.0,0.0,9.0,2.0,7.0,0.0
3,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,3561.0,4.0,1354.0,3.0,0.0,1.0,12.0,2.0,2.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0,0.0,...,2949.0,4.0,3407.0,0.0,0.0,0.0,7.0,2.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,0.0,0.0,0.0,0.0,0.0,148.0,1.0,0.0,0.0,0.0,...,310.0,6.0,2336.0,0.0,0.0,0.0,9.0,3.0,13.0,0.0
34996,0.0,0.0,0.0,0.0,0.0,219.0,1.0,0.0,0.0,0.0,...,2003.0,4.0,227.0,0.0,0.0,0.0,7.0,2.0,7.0,0.0
34997,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,2379.0,4.0,1196.0,0.0,0.0,0.0,10.0,2.0,7.0,0.0
34998,0.0,0.0,0.0,0.0,0.0,111.0,1.0,0.0,0.0,0.0,...,310.0,2.0,2336.0,3.0,0.0,0.0,10.0,3.0,9.0,1.0


In [ ]:
tmp = oe.transform(df_test)
tmp1 = pd.DataFrame(tmp, columns=cols)

In [ ]:
tmp1

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0.0,0.0,0.0,0.0,0.0,1225.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,0.0,0.0,0.0,0.0,0.0,896.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,0.0,0.0,0.0,0.0,0.0,791.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,8.0,0.0,0.0,0.0,0.0,0.0,-1.0,28.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,714.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9996,0.0,0.0,0.0,0.0,0.0,812.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9997,0.0,0.0,0.0,0.0,0.0,819.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9998,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
